# HOW TO obtain catalogue data collected with ESO-VISTA 

The ESO catalogue facility provides access to the collection of catalogues that were produced by PIs of ESO programmes and then integrated into the ESO science archive through the Phase 3 process. The full list of available cataolgues could be found [here](https://www.eso.org/qi/).

In the following, we will show how to collect all catalogues entries associated with the [ESO-VISTA telescope](https://www.eso.org/sci/facilities/paranal/telescopes/vista.html). This will be done by exploiting the `archive_catalogues` module part of the [ESOAsg](https://esoasg.readthedocs.io/en/latest/) `python` repository.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ESOAsg.ancillary import astro
from ESOAsg import archive_catalogues
import matplotlib.pyplot as plt
import numpy as np

/scratch/Extern/anaconda3/envs/py37/lib/python3.7/_collections_abc.py:702: MatplotlibDeprecationWarning: The global colormaps dictionary is no longer considered public API.
  return len(self._mapping)
/scratch/Extern/anaconda3/envs/py37/lib/python3.7/_collections_abc.py:720: MatplotlibDeprecationWarning: The global colormaps dictionary is no longer considered public API.
  yield from self._mapping


In [3]:
%matplotlib notebook

## Obtain information on all catalgues present in the ESO Archive

To access catalogues in the ESO Archive, you can use either the [query interface webpage](https://www.eso.org/qi/), the [ESO Archive Science Portal](https://archive.eso.org/scienceportal/home?dp_type=CATALOG&sort=-obs_date), or the [_programmatic access_](http://archive.eso.org/programmatic/#TAP) via the `tap_cat` TAP Service. 

Alternatively,  you can obtain an astropy table of all catalogues (and all their versions) using:
```python
archive_catalogues.all_catalogues_info(all_versions=True)
```
Note that the column `last_version` highlights with `False` and obsolte catalogue for which a newer version is present. 

It is however possible to remove these obsolete versions by setting:
```python
all_version=False # i.e. the default behavior
```

In [4]:
all_catalogues = archive_catalogues.catalogues_info(all_versions=False)

Now `all_catalogues` contains information on all the latest versions of the catalogues currently present at ESO:

In [5]:
all_catalogues['collection', 'table_name', 'title', 'version', 'instrument', 'telescope'].show_in_notebook(show_row_index=False, display_length=5)

collection,table_name,title,version,instrument,telescope
196.D-0214,EREBOS_RV_cat_fits_V1,EREBOS_RV_cat.fits,1,FORS2,ESO-VLT-U1
196.D-0214,EREBOS_cat_fits_V1,EREBOS_cat.fits,1,FORS2,ESO-VLT-U1
AMBRE,AMBRE_V1,Atmospheric Parameters and Chemical Abundances from Stellar Spectra,1,FEROS,MPI-2.2
ATLASGAL,ATLASGAL_V1,ATLASGAL - APEX Large Area Survey of the Galaxy,1,APEXBOL,APEX-12m
GAIAESO,GES_iDR4_PIII2016_Catalogue_v3_fits_V2,Gaia-ESO spectroscopic survey,2,"GIRAFFE,UVES",ESO-VLT-U2
GCAV,gcav_rxcj1515_YJKs_cat_fits_V1,GCAV catalogue for RXCJ1514.9-1523 cluster,1,VIRCAM,ESO-VISTA
GCAV,gcav_rxcj2129_YJKs_cat_fits_V1,GCAV catalogue for RXCJ2129.6+0005 cluster,1,VIRCAM,ESO-VISTA
GOODS_FORS2,GOODS_FORS2_V1,GOODS/FORS2 Spectroscopic Survey,1,FORS2,UT1
GOODS_ISAAC,GOODS_ISAAC_V1,GOODS/ISAAC imaging,1,ISAAC,ESO-VLT-U1
GOODS_VIMOS_SPEC,GOODS_VIMOS_SPEC_V1,GOODS/VIMOS Spectroscopic Survey,1,VIMOS,ESO-VLT-U3


## Get the catalogues

In the following we will consider the you are interest in data coming from the [ESO-VISTA telescope](https://www.eso.org/sci/facilities/paranal/telescopes/vista.html). Given that `all_catalogues` is a `Table` you can create a `telescope` filter. Possible values are:

In [6]:
print('Possible values for the telescope entry: \n {}'.format(np.unique(all_catalogues['telescope']).tolist()))

Possible values for the telescope entry: 
 ['', 'APEX-12m', 'ESO-NTT', 'ESO-VISTA', 'ESO-VLT-U1', 'ESO-VLT-U2', 'ESO-VLT-U2,ESO-VLT-U3', 'ESO-VLT-U3', 'ESO-VLT-U4', 'ESO-VST', 'ESO-VST, ESO-VISTA', 'MPI-2.2', 'NGTS', 'UT1', 'UT3']


You can now select all the `ESO-VISTA` catalogues applying a filter to the `all_catalogues` table:

In [7]:
telescope_filter = (all_catalogues['telescope'] == 'ESO-VISTA') | (all_catalogues['telescope'] == 'ESO-VST, ESO-VISTA')
vista_catalogues = all_catalogues.copy()[telescope_filter]
vista_catalogues['collection', 'table_name', 'title', 'version', 'instrument', 'telescope'].show_in_notebook(show_row_index=False, display_length=5)

collection,table_name,title,version,instrument,telescope
GCAV,gcav_rxcj1515_YJKs_cat_fits_V1,GCAV catalogue for RXCJ1514.9-1523 cluster,1,VIRCAM,ESO-VISTA
GCAV,gcav_rxcj2129_YJKs_cat_fits_V1,GCAV catalogue for RXCJ2129.6+0005 cluster,1,VIRCAM,ESO-VISTA
KIDS,KiDS_DR4_1_ugriZYJHKs_cat_fits,The Kilo-Degree Survey 9-band ugriZYJHKs source catalogue,4,"OMEGACAM, VIRCAM","ESO-VST, ESO-VISTA"
MW-BULGE-PSFPHOT,MW_BULGE_PSFPHOT_V1,Milky Way Bulge PSF Photometry,1,VIRCAM,ESO-VISTA
UltraVISTA,COSMOS2015_Laigle_v1_1b_latestV7_fits_V1,COSMOS2015 catalogue: photometric redshifts and stellar masses (Laigle et al. 2016),1,VIRCAM,ESO-VISTA
UltraVISTA,UVISTA_5band_cat_dr4_rc_v2_fits_V3,Deep/Ultra-Deep Near-IR Survey of the COSMOS Field (Ultra-VISTA),3,VIRCAM,ESO-VISTA
VEXAS,VEXAS_AllWISE_V1,VISTA extension to auxilliary surveys - match with AllWISE,1,VIRCAM,ESO-VISTA
VHS,VHS_CAT_V3,VISTA Hemisphere Survey band-merged multi-waveband catalogues (VHS),3,VIRCAM,ESO-VISTA
VIDEO,video_er3_zyjhks_CDFS_catMetaData_fits_V2,VISTA Deep Extragalactic Observations Survey (VIDEO) - CDFS field,2,VIRCAM,ESO-VISTA
VIDEO,video_er3_zyjhks_ES1_catMetaData_fits_V2,VISTA Deep Extragalactic Observations Survey (VIDEO) - ES1 field,2,VIRCAM,ESO-VISTA


To obtain the description of each table, you can recursively run over the `description` column:

In [8]:
for idx in range(len(vista_catalogues)):
    print('Collection:  {} \nTable:       {} \nDescription: {} \n \n'.format(vista_catalogues['collection'][idx],
                                                                       vista_catalogues['table_name'][idx],
                                                                       vista_catalogues['description'][idx]))

Collection:  GCAV 
Table:       gcav_rxcj1515_YJKs_cat_fits_V1 
Description: GCAV YJKs merged catalogue for RXCJ1514.9-1523 cluster, containing aperture matched photometry from the deep stacks 
 

Collection:  GCAV 
Table:       gcav_rxcj2129_YJKs_cat_fits_V1 
Description: GCAV YJKs merged catalogue for RXCJ2129.6+0005 cluster, containing aperture matched photometry from the deep stacks 
 

Collection:  KIDS 
Table:       KiDS_DR4_1_ugriZYJHKs_cat_fits 
Description: The DR4.1 catalogue is an update of the DR4.0 catalogue which, due to a small bug in the alignment of the mask images during the multi-band catalogue production, has incorrect photometry flags for a small percentage of sources on 196 tiles. 
 

Collection:  MW-BULGE-PSFPHOT 
Table:       MW_BULGE_PSFPHOT_V1 
Description: This catalog provides a comprehensive census of the stellar populations in the inner ~300 sqdeg of the Galaxy. It is based on observations obtained with VISTA as part of the ESO public survey VVV. The measu

## Identify RA, Dec, and Source ID

Now you are ready the recursively explore all catalogues containing data collected with the [ESO-VISTA telescope](https://www.eso.org/sci/facilities/paranal/telescopes/vista.html). There are however few points that need to be taken into account first.

Some catalogues are spread among different tables. This is the case, for instance, of the [VISTA survey of the Magellanic Clouds system (VMC)](https://www.eso.org/rm/api/v1/public/releaseDescriptions/155). The reference catalogue that contains position and ID of the sources is identified by the columns`target_table` and `target_column`

In [9]:
vmc_filter = (vista_catalogues['collection'] == 'VMC')
vmc_catalogues = vista_catalogues.copy()[vmc_filter]
vmc_catalogues['collection', 'table_name', 'table_ID', 'target_table', 'target_column'].show_in_notebook(show_row_index=False, display_length=5)

collection,table_name,table_ID,target_table,target_column
VMC,vmc_dr5_mPhotJ_V4,SOURCEID,vmc_dr5_sourceCat_yjks_V4,SOURCEID
VMC,vmc_dr5_mPhotKs_V4,SOURCEID,vmc_dr5_sourceCat_yjks_V4,SOURCEID
VMC,vmc_dr5_mPhotY_V4,SOURCEID,vmc_dr5_sourceCat_yjks_V4,SOURCEID
VMC,vmc_dr5_psf_yjks_V3,PSFSOURCEID,vmc_dr5_sourceCat_yjks_V4,SOURCEID
VMC,vmc_dr5_sourceCat_yjks_V4,SOURCEID,,
VMC,vmc_dr5_var_yjKs_V1,SOURCEID,vmc_dr5_sourceCat_yjks_V4,SOURCEID
VMC,vmc_er4_yjks_cepheidCatMetaData_fits_V3,VARID,vmc_er4_ksjy_catMetaData_fits_V3,SOURCEID
VMC,vmc_er4_yjks_eclBinCatMetaData_fits_V2,VARID,vmc_er4_ksjy_catMetaData_fits_V3,SOURCEID
VMC,vmc_er4_yjks_rrLyrCatMetaData_fits_V1,VARID,vmc_er4_ksjy_catMetaData_fits_V3,SOURCEID


This basically means, for instance, that basic information on the targets present in the table `vmc_dr5_mPhotJ_V4` are located in the table `vmc_dr5_sourceCat_yjks_V4` and the connection is based on the columns `SOURCEID`. These are identified by the `ucd` token `meta.id;meta.main`.

In [10]:
columns_VMC_PhotJ = archive_catalogues.columns_info(tables='vmc_dr5_mPhotJ_V4')
columns_VMC_PhotJ[(columns_VMC_PhotJ['ucd'] == 'meta.id;meta.main')].show_in_notebook(show_row_index=False)

table_name,column_name,ucd,datatype,description,unit
vmc_dr5_mPhotJ_V4,SOURCEID,meta.id;meta.main,BIGINT,UID of this merged detection,


In [11]:
columns_VMC_Sources = archive_catalogues.columns_info(tables='vmc_dr5_sourceCat_yjks_V4')
columns_VMC_Sources[(columns_VMC_Sources['ucd'] == 'meta.id;meta.main')].show_in_notebook(show_row_index=False)

table_name,column_name,ucd,datatype,description,unit
vmc_dr5_sourceCat_yjks_V4,SOURCEID,meta.id;meta.main,BIGINT,UID of this merged detection as assigned by merge algorithm,


## Explore catalogue content

You can glance the content of a catalogue by checking the name of the columns present in the associated list of collections (or list of tables). For instance, for the [Kilo Degree Survey](https://www.eso.org/rm/api/v1/public/releaseDescriptions/151)
```python
archive_catalogues.columns_info(tables='KiDS_DR4_1_ugriZYJHKs_cat_fits')
```

In [12]:
table_kids = 'vmc_dr5_mPhotJ_V4'
table_kids = 'vmc_dr5_psf_yjks_V3'
columns_kids = archive_catalogues.columns_info(tables=table_kids)
columns_kids[0:100].show_in_notebook(show_row_index=False, display_length=5)

table_name,column_name,ucd,datatype,description,unit
vmc_dr5_psf_yjks_V3,IAUNAME,meta.id,CHAR,IAU Name (not unique),
vmc_dr5_psf_yjks_V3,SOURCEID,meta.id,BIGINT,UID of this merged detection as assigned by merge algorithm,
vmc_dr5_psf_yjks_V3,DISTANCEMINS,pos.angDistance,REAL,Angular separation between neighbours,arcminutes
vmc_dr5_psf_yjks_V3,PSFSOURCEID,meta.id;meta.main,BIGINT,UID of VMC PSF extracted objects,
vmc_dr5_psf_yjks_V3,FIELDNAME,meta.id,CHAR,ID of field,
vmc_dr5_psf_yjks_V3,FRAMESETID,meta.id,BIGINT,"frame set ID, linked to vmcMergeLog, assigned by merging procedure",
vmc_dr5_psf_yjks_V3,CUEVENTID,meta.id,INTEGER,UID of curation event giving rise to this record,
vmc_dr5_psf_yjks_V3,RAY,pos.eq.ra;em.IR.NIR,DOUBLE,PSF fit RA centre Y filter,deg
vmc_dr5_psf_yjks_V3,DECY,pos.eq.dec;em.IR.NIR,DOUBLE,PSF fit Dec centre Y filter,deg
vmc_dr5_psf_yjks_V3,YPSFMAG,stat.fit.param;phot.mag;em.IR.NIR;meta.main,REAL,3 pixels PSF fitting magnitude Y filter,mag


Once you decided which is your favourite catalogue, you can get it using:
```python
archive_catalogues.get_catalogues(tables=table_name)
```
At the moment with limit the query to only 2 entries of `KIDS` (i.e. we set `maxrec=2`):

In [ ]:
KIDS = archive_catalogues.get_catalogues(tables=table_kids, maxrec=2, join_target_table=True)

[INFO]    :: The query is:
[INFO]    :: 
            SELECT 
                SOURCEID, RA2000, DEC2000 
            FROM 
                vmc_dr5_sourceCat_yjks_V4
            
            WHERE
                (
                SOURCEID = 558377355612
                )


Python 3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 23:03:20) 
Type 'copyright', 'credits' or 'license' for more information
IPython 7.17.0 -- An enhanced Interactive Python. Type '?' for help.

In [1]:                 query_target_table.run_query(to_string=True)

In [2]:                 target_catalogue = query_target_table.get_result_from_query()



The result is thus stored in an `astropy.table`:

In [ ]:
vista_catalogues['table_name', 'target_table', 'target_column',
    'last_version', 'table_ID', 'table_RA', 'table_Dec'].show_in_notebook(show_row_index=False, display_length=5)

In [ ]:
KIDS['ID', 'RAJ2000', 'DECJ2000', 'MAG_GAAP_Y', 'MAG_GAAP_J', 'MAG_GAAP_H'].show_in_notebook(show_row_index=False, display_length=2)

In [ ]:
for idx in range(len(vista_catalogues)):
    print('Collection:  {} \nTable:       {} \nDescription: {} \n \n'.format(vista_catalogues['RA_id'][idx],
                                                                       vista_catalogues['table_name'][idx],
                                                                       vista_catalogues['description'][idx]))

In [ ]:
for idx in range(len(vista_catalogues)):
    table_name =  vista_catalogues['table_name'][idx]
    target_table =  vista_catalogues['target_table'][idx]
    if target_table is '':
        print(table_name)
        print(target_table)
        columns = archive_catalogues.columns_info(tables=table_name)
        ra_name = columns['column_name'][ (columns['ucd'] == 'pos.eq.ra;meta.main') ][0]
        de_name = columns['column_name'][ (columns['ucd'] == 'pos.eq.dec;meta.main') ][0]    
        print(ra_name, de_name)
        print(" ")



"""
VIKING = archive_catalogues.get_catalogues(tables=table_viking, columns=selected_columns, maxrec=20000,
                                          type_of_query='async')

CLASS_STAR = VIKING['MERGEDCLASS'].data
JK = VIKING['JAPERMAG3'].data-VIKING['KSAPERMAG3'].data
ZJ = VIKING['ZAPERMAG3'].data-VIKING['JAPERMAG3'].data

plt.figure()
plt.xlim(-.3,2.5)
plt.ylim(-.3,2.3)
plt.scatter(JK[CLASS_STAR==-1],ZJ[CLASS_STAR==-1], label=r'STAR', c='dodgerblue', alpha=0.2, marker='.')
plt.scatter(JK[CLASS_STAR==1],ZJ[CLASS_STAR==1], label=r'EXTENDED', c='crimson', alpha=0.2, marker='.')
plt.xlabel(r'$J - K_s$')
plt.ylabel(r'$Z - J$')
plt.legend()
plt.show()
plt.savefig('viking.jpg', format='jpg', quality=99, bbox_inches='tight')
"""